In [ ]:
from actors import Vehicle, Pedestrian
from typing import Optional, List
import random


In [ ]:
import carla

In [ ]:
def get_client() -> carla.libcarla.Client:
    """
    Get a CARLA client object. The client object enables interaction with the CARLA server.
    """
    client = carla.Client("localhost", 2000)
    client.set_timeout(10.0)
    print("Connection to CARLA server established!")
    return client


In [ ]:
# Get client to interact with CARLA server
client = get_client()

# Get current CARLA world
world = client.get_world()

In [ ]:
def spawn_vehicle(
    world: carla.libcarla.World,
    vehicle_type: str = "model3",
    spawn_point: Optional[carla.libcarla.Transform] = None,
) -> carla.libcarla.Vehicle:
    """
    Spawns a vehicle at a given spawn point. Default car model is 'model3'.
    If no spawn point is provided, randomly selects the spawn point from the set of pre-assigned spawn points in the map.
    """

    if spawn_point is None:
        spawn_point = random.choice(world.get_map().get_spawn_points())
    vehicle_blueprint = world.get_blueprint_library().filter(vehicle_type)[0]
    vehicle = world.spawn_actor(vehicle_blueprint, spawn_point)
    return vehicle


In [ ]:
all_waypoints = world.get_map().generate_waypoints(1)
def filter_waypoints(waypoints, road_id, lane_id = None):
    
    filtered_waypoints = []
    for wp in waypoints:
        
        if lane_id == None and wp.road_id == road_id:
            filtered_waypoints.append(wp) 
        elif wp.road_id == road_id and wp.lane_id==lane_id:
            filtered_waypoints.append(wp) 

    return filtered_waypoints


def spawn_vehicle(
    world: carla.libcarla.World,
    vehicle_type: str = "model3",
    spawn_point: Optional[carla.libcarla.Transform] = None,
) -> carla.libcarla.Vehicle:
    """
    Spawns a vehicle at a given spawn point. Default car model is 'model3'.
    If no spawn point is provided, randomly selects the spawn point from the set of pre-assigned spawn points in the map.
    """

    if spawn_point is None:
        spawn_point = random.choice(world.get_map().get_spawn_points())
    vehicle_blueprint = world.get_blueprint_library().filter(vehicle_type)[0]
    vehicle = world.spawn_actor(vehicle_blueprint, spawn_point)
    return vehicle


def draw_waypoints(
    world: carla.libcarla.World,
    waypoints: List[carla.libcarla.Waypoint],
    life_time: float = 30.0,  # Seconds
) -> None:
    """
    Draws a list of waypoints in the given CARLA world.
    """
    for waypoint in waypoints:
        world.debug.draw_string(
            waypoint.transform.location,
            "O",
            draw_shadow=False,
            color=carla.Color(r=0, g=255, b=0),
            life_time=life_time,
            persistent_lines=True,
        )


In [ ]:
wps = filter_waypoints(all_waypoints, road_id=12)



In [ ]:
vehicle_blueprint = world.get_blueprint_library().filter('walker.pedestrian.*')[10]
ped1 = world.spawn_actor(vehicle_blueprint, wps[15].transform)
ped2 = world.spawn_actor(vehicle_blueprint, wps[10].transform)
ped3 = world.spawn_actor(vehicle_blueprint, wps[5].transform)

In [ ]:
wp1 = np.random.choice(wps, 10)
# draw_waypoints(world, wp1)

In [ ]:
world.get_blueprint_library().filter('model3*')

In [ ]:
actors = []
for i,wp in enumerate(wp1):
    
    x = wp.transform.location.x
    y = wp.transform.location.y
    z = wp.transform.location.z + 0.5
    yaw = wp.transform.rotation.yaw
    transform = carla.Transform(carla.Location(x=x,y=y,z=z), carla.Rotation(yaw=yaw))

    actors.append(spawn_vehicle(world, 'model3', transform))
    

In [ ]:
for actors in world.get_actors().filter('*'):
    actors.destroy()

In [ ]:
wp1[0].transform.location.y, actors[0].get_location().y

In [ ]:
draw_waypoints(world, [wp1[0]])

In [ ]:
our_vehicle = Vehicle(world, actors[0].id)

In [ ]:
closest_vehicles, idxs = Vehicle.getClosest([Vehicle(world, actor.id) for actor in actors[1:]], our_vehicle, 5)

In [ ]:
closest_pedestrians, idxs = Pedestrian.getClosestPedestrian([Pedestrian(world, ped.id) for ped in [ped1, ped2, ped3]], our_vehicle, 1)

In [ ]:
our_vehicle = Vehicle(world, actors[0].id)

In [ ]:
vehicle_location = carla.Location(x=our_vehicle.location["x"], y=our_vehicle.location["y"], z=our_vehicle.location["z"])
nearest_waypoint = world.get_map().get_waypoint(
    vehicle_location, project_to_road=True
)
draw_waypoints(world, [nearest_waypoint])

In [ ]:
nearest_waypoint.

In [ ]:
for vehicle in closest_vehicles + [our_vehicle]:
    vehicle_location = carla.Location(x=vehicle.location["x"], y=vehicle.location["y"], z=vehicle.location["z"])
    nearest_waypoint = world.get_map().get_waypoint(
        vehicle_location, project_to_road=True
    )
    draw_waypoints(world, [nearest_waypoint])

In [ ]:
draw_waypoints(world, [nearest_waypoint])